# Assignment 5
### prasmus3

## Import required modules and define functions

In [1]:
from typing import Any, List, Dict
import numpy as np
import pandas as pd


toy_data = pd.DataFrame([
    dict(a=1, b="zounds", c=np.nan),
    dict(a=2, b="gronks", c=1),
    dict(a=3, b="sounds", c=2),
    dict(a=-2, b="zounds", c=4),
])

## Functions

In [2]:
def compute_n_cats(data: pd.DataFrame) -> pd.Series:
    """
    Compute number of unique categories for each categorical column.
    Args:
        data: Dataframe with categorical features.
 
    Returns: List of each categorical feature with count of unique categories.

    """
    categorical_feats = list(data.select_dtypes("object"))
    counts = []
    for categorical_feat in categorical_feats:
        counts.append(len(data[categorical_feat].unique()))
    return pd.Series(counts, index=categorical_feats, name="cat_counts")


assert compute_n_cats(toy_data).name == "cat_counts"
assert compute_n_cats(toy_data).index[0] == "b"

In [3]:
def compute_proportions(series: pd.Series) -> pd.DataFrame:
    """
    Compute count, fraction, and cumulative fraction of categories.
    Args:
        series: Pandas series of categorical variables.
    Returns: Count, fraction, & cum. fraction (descending order) of series.

    """
    value_counts = series.copy().value_counts().rename("count").to_frame()
    value_counts["fraction"] = value_counts["count"] / value_counts["count"].sum()
    value_counts["cum_frac"] = value_counts["fraction"].cumsum()
    return value_counts


assert compute_proportions(toy_data["b"]).shape == (3, 3)
assert np.allclose(compute_proportions(toy_data["b"])["fraction"].sum(), 1)

In [4]:
def dummy_nominals(data: pd.DataFrame, nominal_cols: List[str]=[], dummy_kwargs: Dict={}) -> pd.DataFrame:
    data_ = data.copy()
    dummies = []
    for nominal_col in nominal_cols:
        dummy_kwargs.update(dict(prefix=nominal_col))
        dummies.append(pd.get_dummies(data_[nominal_col], **dummy_kwargs))
    if len(dummies) > 0:
        dummies = pd.concat(dummies, axis=1)
        return data_.drop(axis=1, labels=nominal_cols).join(dummies)
    return data_


assert "b_gronks" in dummy_nominals(toy_data, nominal_cols=["b"])
assert dummy_nominals(toy_data).equals(toy_data)

In [5]:
def fill_nans(data: pd.DataFrame, fill_dict: Dict[str, Any]={}):
    """
    Fill NaN / missing values using mean, median, drop, or supplied value.
    Args:
        data: Data with columns with missing data.
        fill_dict: Dictionary keyed by column with method or value.
    Returns: Dataframe with missing values filled.

    """
    # Fill missing data
    data_ = data.copy()
    for col, fill in fill_dict.items():
        if fill == "median":
            fill_value = data[col].median()
        elif fill == "mean":
            fill_value = data[col].mean()
        elif isinstance(fill, str):
            raise NotImplementedError(f"Method {fill} not supported at this time.")
        else:
            fill_value = fill
        data_[col] = data_[col].fillna(fill_value)
    return data_


assert np.isnan(fill_nans(toy_data).loc[0, "c"])
assert fill_nans(toy_data, fill_dict=dict(c=10)).loc[0, "c"] == 10.0
assert fill_nans(toy_data, fill_dict=dict(c="median")).loc[0, "c"] == 2.0

In [6]:
def get_nulls(data: pd.DataFrame) -> pd.Series:
    """
    Get columns with null values and their counts.
    Args:
        data: Dataframe which may have nulls.
    Returns: Series with zero or more columns and associated counts.
 
    """
    nulls = data.isnull().sum()
    return nulls[nulls>0]


assert get_nulls(toy_data).shape == (1,)
assert get_nulls(toy_data).index[0] == "c"

In [7]:
def preprocess(
        data: pd.DataFrame,
        fill_dict: Dict[str, Any]={},
        nominal_cols: List[str]=[],
        ordinal_cols: List[str]=[],
        drop_cols: List[str]=[],
        drop_obs: bool=False,
        drop_kwargs: Dict[str, Any]={},
        dummy_kwargs: Dict[str, Any]={},
) -> pd.DataFrame:
    """
    Ready data for use in deep neural network: drop columns, drop observations / impute missing
    values, and dummy categorical data.
    Args:
        data: Dataframe to preprocess.
        fill_dict: Dictionary of fill parameters; not used if drop_obs is not empty.
        nominal_cols: Nominal columns to dummy.
        ordinal_cols: Ordinal columns to dummy; this is currently not implemented.
        drop_cols: List of columns to drop.
        drop_obs: True to drop observations using criteria specied in drop_kwargs.
        drop_kwargs: Provide keyword arguments for Pandas dropna method.
    """
    # Raise NotImplementedError if user specifies ordinal columns to dummy
    if ordinal_cols:
        raise NotImplementedError("Dummying of ordinal columns not supported at this time.")
    # Drop columns
    data_ = data.copy().drop(axis=1, labels=drop_cols)
    # Dropping observations and imputation are mutually exclusive in this implementation
    if drop_obs:
        data_ = data_.dropna(axis=0,  **drop_kwargs)
    # Fill missing data
    else:
        data_ = fill_nans(data_, fill_dict)
    # Dummy nominal columns
    data_ = dummy_nominals(data_, nominal_cols, dummy_kwargs)
    return data_


assert np.isnan(preprocess(toy_data).loc[0].loc["c"])
assert preprocess(toy_data, fill_dict=dict(c=-100)).loc[0].loc["c"] == -100
assert preprocess(toy_data, drop_obs=True, drop_kwargs=dict(how="any")).shape == (3, 3)
assert preprocess(toy_data, nominal_cols=["b"], dummy_kwargs=dict(drop_first=True)).shape == (4, 4)

## Download data

In [8]:
!wget -o cars.csv https://github.com/pgr-me/data/tree/main/aisys/module5/cars.csv

## Read data and print feature datatypes

In [9]:
raw = pd.read_csv("https://github.com/pgr-me/data/raw/main/aisys/module5/cars.csv")
print(raw.info())
raw.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38531 entries, 0 to 38530
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   manufacturer_name  38531 non-null  object 
 1   model_name         38531 non-null  object 
 2   transmission       38531 non-null  object 
 3   color              38531 non-null  object 
 4   odometer_value     38531 non-null  int64  
 5   year_produced      38531 non-null  int64  
 6   engine_fuel        38531 non-null  object 
 7   engine_has_gas     38531 non-null  bool   
 8   engine_type        38531 non-null  object 
 9   engine_capacity    38521 non-null  float64
 10  body_type          38531 non-null  object 
 11  has_warranty       38531 non-null  bool   
 12  state              38531 non-null  object 
 13  drivetrain         38531 non-null  object 
 14  price_usd          38531 non-null  float64
 15  is_exchangeable    38531 non-null  bool   
 16  location_region    385

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,...,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,Subaru,Outback,automatic,silver,190000,2010,gasoline,False,gasoline,2.5,...,True,True,True,False,True,False,True,True,True,16
1,Subaru,Outback,automatic,blue,290000,2002,gasoline,False,gasoline,3.0,...,True,False,False,True,True,False,False,False,True,83
2,Subaru,Forester,automatic,red,402000,2001,gasoline,False,gasoline,2.5,...,True,False,False,False,False,False,False,True,True,151
3,Subaru,Impreza,mechanical,blue,10000,1999,gasoline,False,gasoline,3.0,...,False,False,False,False,False,False,False,False,False,86
4,Subaru,Legacy,automatic,black,280000,2001,gasoline,False,gasoline,2.5,...,True,False,True,True,False,False,False,False,True,7


## Look at number of categories
* We see that `model_name`, a nominal categorical feature, has too many categories.
* We can either:
  * Drop the column
  * Reduce categories
  * Engineer additional features, each with reduced category number

In [10]:
compute_n_cats(raw)

manufacturer_name      55
model_name           1118
transmission            2
color                  12
engine_fuel             6
engine_type             3
body_type              12
state                   3
drivetrain              3
location_region         6
Name: cat_counts, dtype: int64

### Let's look at `model_name` in greater detail
* We see that no one model dominates the categories; the most frequent cateogry makes up less thatn 4% of total records.
* The distribution has a long tail.
* Reducing categories through simplification for such heterogenous categories seems iffy.
* Engineering new features each with fewer categories seems unnecessarily onerous and it's unknown if we'd realize any predictive benefit.
* As such, let's eliminate the feature.
* If we had more time, we could add an external dataset and join on model. What if that external dataset included vehicle weight, vehicle miles per gallon, and other features that would be of interest? Although we wouldn't use model in our final analysis we'd use it as an intermediate key to get features that we would include in our final feature set.

In [11]:
compute_proportions(raw.model_name.str.lower().str.strip()).head(20)

,count,fraction,cum_frac
passat,1423,0.036931,0.036931
astra,751,0.019491,0.056422
golf,707,0.018349,0.074771
a6,687,0.017830,0.092601
mondeo,637,0.016532,0.109133
vectra,565,0.014664,0.123796
laguna,548,0.014222,0.138019
a4,505,0.013106,0.151125
406,415,0.010771,0.161896
omega,387,0.010044,0.171939


### Let's add `model_name` to our list of columns to drop

In [12]:
drops = ["model_name"] 

# Nominal versus ordinal data
* We know we have categorical data that we must prepare, but are there any ordinal features?
* I don't see any obvious ones in the dataset. You could make an argument that `body_type`, which correlates with weight, could be coerced to an ordinal. But then how do we know that, say hatchback, is a 0 and coup is 1? And sedan is 2, etc. Rather than force it, I'm going to just treat `body_type` as nominal.
* The same goes for `drivetrain` (forward, reverse, all) and `engine_type`. 
* `color` is obviously nominal (colors aren't ordered) nor are the location fields nor the manufacturer.
* So then, let's treat all the categorical features as nominal and one-hot-encode them, dropping the first category. 

In [13]:
nominal_cols = [x for x in list(raw.select_dtypes(include=["object"])) if x not in drops]
ordinal_cols = []
print(f"Nominal columns: {nominal_cols}")
print(f"Ordinal columns: {ordinal_cols}")

Nominal columns: ['manufacturer_name', 'transmission', 'color', 'engine_fuel', 'engine_type', 'body_type', 'state', 'drivetrain', 'location_region']
Ordinal columns: []


### Null values
* We need to handle null values, either by imputation or removal.
* Let's see how many columns have null values and the number of nulls in each.

In [14]:
null_columns = get_nulls(raw)
print("Null columns and counts")
print(null_columns)

Null columns and counts
engine_capacity    10
dtype: int64


Wow, only one column, `engine_capacity`, has ten nulls out of 38,531 records? We *could* just remove the offending rows, but let's just look at the rows first before doing so.

In [15]:
raw[raw.engine_capacity.isnull()]

,manufacturer_name,model_name,transmission,color,odometer_value,year_produced,engine_fuel,engine_has_gas,engine_type,engine_capacity,...,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
8782,Fiat,500,automatic,orange,27000,2013,electric,False,electric,NaN,...,True,False,True,True,True,False,True,True,True,77
9048,Fiat,500,automatic,orange,49000,2014,electric,False,electric,NaN,...,False,False,True,False,True,False,True,False,True,11
24226,Chevrolet,Volt,automatic,silver,168000,2013,electric,False,electric,NaN,...,False,False,True,False,False,False,True,True,True,6
25943,Nissan,Leaf,automatic,white,57357,2015,electric,False,electric,NaN,...,True,True,True,True,True,True,True,True,True,75
26203,Nissan,Leaf,automatic,blue,97400,2011,electric,False,electric,NaN,...,True,False,False,False,False,False,True,False,True,64
26222,Nissan,Leaf,automatic,white,50000,2014,electric,False,electric,NaN,...,True,False,False,False,True,False,True,True,False,18
26582,Nissan,Leaf,automatic,black,84000,2014,electric,False,electric,NaN,...,False,False,False,False,True,True,True,True,True,138
26914,Nissan,Leaf,automatic,black,84500,2013,electric,False,electric,NaN,...,True,False,True,False,True,True,True,True,True,58
27554,BMW,i3,automatic,white,54150,2015,electric,False,electric,NaN,...,True,True,True,False,True,True,True,True,True,18
29590,BMW,i3,automatic,other,67000,2018,electric,False,electric,NaN,...,True,True,True,True,True,True,True,True,True,57


Oh no, it turns out *all* of our electric vehicles have a null `engine_capacity`. Yes, these ten rows are a drop in the bucket of total observations, but if we eliminate them we lose *all* of our electric vehicles. We see that these rows are uniquely identified by the `engine_fuel` feature, which is `electric` for every row. No other observation has `electric` for `engine_feature`.

Using a procedure such as `MICE` for imputation for ten observations seems like overkill, so let's not do that.

We know that we're preparing our data for a deep learning network, and we know deep learning networks handle non-linearity well. Let's say we fill the null `engine_feature` values with a number. A deep learning network should have sufficient depth to separate the electric from non-electric vehicles and then be able to operate on the originally non-null `engine_feature` values. Let's **assume** so for our analysis. 

Let's fill the null `engine_capacity` field with the median of that field's non-null values.

In [16]:
print("engine_capacity summary statistics, including minimum and maximum, which bookend the domain of that feature's values.")
raw.engine_capacity.describe()

engine_capacity summary statistics, including minimum and maximum, which bookend the domain of that feature's values.


count    38521.000000
mean         2.055161
std          0.671178
min          0.200000
25%          1.600000
50%          2.000000
75%          2.300000
max          8.000000
Name: engine_capacity, dtype: float64

In [17]:
fill_values = dict(engine_capacity="median")
print(f"Fill dictionary: {fill_values}")

Fill dictionary: {'engine_capacity': 'median'}


## One-hot encode / dummy categorical features
* We already decided that all the features are nominal so that makes this exerciese pretty straightforward.
* We also identified a feature we need to impute and decided on our imputation strategy.
* We can proceed with transforming the data to prepare it for the deep learning network.


In [18]:
processed = preprocess(raw, fill_dict=fill_values, drop_cols=drops, nominal_cols=nominal_cols, )
processed.to_csv("processed.csv")
processed

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,price_usd,is_exchangeable,number_of_photos,up_counter,feature_0,...,state_owned,drivetrain_all,drivetrain_front,drivetrain_rear,location_region_Брестская обл.,location_region_Витебская обл.,location_region_Гомельская обл.,location_region_Гродненская обл.,location_region_Минская обл.,location_region_Могилевская обл.
0,190000,2010,False,2.5,False,10900.00,False,9,13,False,...,1,1,0,0,0,0,0,0,1,0
1,290000,2002,False,3.0,False,5000.00,True,12,54,False,...,1,1,0,0,0,0,0,0,1,0
2,402000,2001,False,2.5,False,2800.00,True,4,72,False,...,1,1,0,0,0,0,0,0,1,0
3,10000,1999,False,3.0,False,9999.00,True,9,42,True,...,1,1,0,0,0,0,0,0,1,0
4,280000,2001,False,2.5,False,2134.11,True,14,7,False,...,1,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38526,290000,2000,False,3.5,False,2750.00,True,5,85,False,...,1,0,1,0,0,0,0,0,1,0
38527,321000,2004,False,2.2,False,4800.00,True,4,20,False,...,1,0,1,0,1,0,0,0,0,0
38528,777957,2000,False,3.5,False,4300.00,False,3,63,False,...,1,0,1,0,0,0,0,0,1,0
38529,20000,2001,False,2.0,False,4000.00,True,7,156,False,...,1,0,1,0,1,0,0,0,0,0


## Data types of transformed dataset

We see that each feature is numeric and can be used in the deep learning network.

In [19]:
processed.dtypes

odometer_value                        int64
year_produced                         int64
engine_has_gas                         bool
engine_capacity                     float64
has_warranty                           bool
                                     ...   
location_region_Витебская обл.        uint8
location_region_Гомельская обл.       uint8
location_region_Гродненская обл.      uint8
location_region_Минская обл.          uint8
location_region_Могилевская обл.      uint8
Length: 122, dtype: object